In [2]:
import openai
import pandas as pd
import numpy as np
import json
import os
from scipy import stats
from scipy.stats import chi2_contingency

In [3]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/Categorization_AI_experiments


In [6]:
Main = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/e1b/e1ab_Master.xlsx", engine='openpyxl', sheet_name="Master_coded").drop(["prim_key"], axis=1)
Main = Main.iloc[:400, :22]
Main
#Main.columns = range(Main.shape[1])

Chris = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/e1b/e1b_Master.xlsx", engine='openpyxl', sheet_name="CS_Categorization")
Chris = Chris.iloc[:400, :22]
#Nigel.columns = range(Nigel.shape[1])

Claude = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/e1b/e1b_Master.xlsx", engine='openpyxl', sheet_name="CF_Categorization")
Claude = Claude.iloc[:400, :22]
#Rebeca.columns = range(Rebeca.shape[1])

bad = pd.read_csv('data/e1b/e1b_bad_categorization_20_cats.csv').drop(['link1','json'], axis=1)
bad = bad.rename(columns = {"survey_response": "Response"})

good = pd.read_csv('data/e1b/e1b_good_categorization_20_cats.csv').drop(['link1','json'], axis=1)
good = good.rename(columns={"survey_response": "Response"})

cot = pd.read_csv('data/e1b/e1b_cot_categorization_20_cats.csv').drop(['link1','json'], axis=1)
cot = cot.rename(columns={"survey_response": "Response"})

#cove = pd.read_csv('data/e1b/e1b_cove_categorization_20_cats.csv').drop(['survey_response','link1','link2','json'], axis=1)
oneshot = pd.read_csv('data/e1b/e1b_1s_categorization_20_cats.csv').drop(['link1','json'], axis=1)
oneshot = oneshot.rename(columns={"survey_response": "Response"})

fewshot = pd.read_csv('data/e1b/e1b_fs_categorization_20_cats.csv').drop(['link1','json'], axis=1)
fewshot = fewshot.rename(columns={"survey_response": "Response"})

Main

KeyError: "['prim_key'] not found in axis"

Rebeca.fillna(0, inplace=True)
Rebeca['Response'] = Rebeca['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)
Main['Response'] = Main['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)

Rebeca = Rebeca.astype(int, errors='ignore')

Main_Rebeca = pd.merge(Main, Rebeca, on = "Response", how = "left")

Nigel.fillna(0, inplace=True)
Nigel['Response'] = Nigel['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)
Main['Response'] = Main['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)

Nigel = Nigel.astype(int, errors='ignore')

Main_Nigel = pd.merge(Main, Nigel, on = "Response", how = "left")

Main_Rebeca.to_csv('data/a19f/Rebeca.csv')
Main_Nigel.to_csv('data/a19f/Nigel.csv')

bad.fillna(0, inplace=True)
bad['Response'] = bad['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)

bad = bad.astype(int, errors='ignore')

Main_bad = pd.merge(Main, bad, on = "Response", how = "left")

Main_bad.to_csv('data/a19f/Bad.csv')

good.fillna(0, inplace=True)

good['Response'] = good['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)

good = good.astype(int, errors='ignore')

Main_good = pd.merge(Main, good, on="Response", how="left")

Main_good.to_csv('data/a19f/Good.csv')


cot.fillna(0, inplace=True)


cot['Response'] = cot['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)


cot = cot.astype(int, errors='ignore')


Main_cot = pd.merge(Main, cot, on="Response", how="left")

Main_cot.to_csv('data/a19f/COT.csv')

oneshot.fillna(0, inplace=True)

oneshot['Response'] = oneshot['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)

oneshot = oneshot.astype(int, errors='ignore')

Main_oneshot = pd.merge(Main, oneshot, on="Response", how="left")

Main_oneshot.to_csv('data/a19f/oneshot.csv')

fewshot.fillna(0, inplace=True)

fewshot['Response'] = fewshot['Response'].str.strip().str.replace(r'\s+', ' ', regex=True)

fewshot = fewshot.astype(int, errors='ignore')

Main_fewshot = pd.merge(Main, fewshot, on="Response", how="left")

Main_fewshot.to_csv('data/a19f/fewshot.csv')

I cleaned this up manually to fill in some gaps that didn't match

In [62]:
Rebeca = pd.read_csv('data/a19f/Rebeca.csv').drop(['Response'], axis=1)
Nigel = pd.read_csv('data/a19f/Nigel.csv').drop(['Response'], axis=1)
bad = pd.read_csv('data/a19f/Bad.csv').drop(['Response'], axis=1)
good = pd.read_csv('data/a19f/Good.csv').drop(['Response'], axis=1)
cot = pd.read_csv('data/a19f/COT.csv').drop(['Response'], axis=1)
oneshot = pd.read_csv('data/a19f/oneshot.csv').drop(['Response'], axis=1)
fewshot = pd.read_csv('data/a19f/fewshot.csv').drop(['Response'], axis=1)

Main = Main.drop(['Response'], axis=1)
Rebeca.head()

,Religious Community Involvement_y,Local Venue Visitation_y,Secular Volunteer Service_y,Online Social Networking_y,Workplace Socializing_y,Athletic Engagement_y,Cultural-Hobby Participation_y,Neighborhood Integration_y,Broader Event and Group Participation_y
0,0,0,0,0,0,1,1,0,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,1


In [63]:
cot

,1,2,3,4,5,6,7,8,9
0,0,0,1,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
395,0,0,1,0,0,0,0,0,1
396,0,0,0,0,0,0,0,0,0
397,0,0,0,0,0,0,0,0,0
398,0,0,0,0,1,0,0,1,1


In [64]:
fewshot

,1,2,3,4,5,6,7,8,9
0,0,0,1,0,0,1,0,1,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
395,0,0,1,0,0,0,0,0,1
396,0,0,0,0,0,0,0,0,0
397,0,0,0,0,0,0,0,0,0
398,0,0,0,0,1,0,0,1,0


In [65]:
bad

,1,2,3,4,5,6,7,8,9
0,0,0,1,0,0,1,0,1,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
395,0,0,1,0,0,0,0,0,1
396,0,0,0,0,0,0,0,0,0
397,0,0,0,0,0,0,0,0,0
398,0,0,0,0,1,0,0,1,0


In [66]:
print(f"The dataframe Main has {len(Main)} rows.")

print(f"The dataframe bad has {len(bad)} rows.")

print(f"The dataframe good has {len(good)} rows.")

print(f"The dataframe cot has {len(cot)} rows.")

#print(f"The dataframe cove has {len(cove)} rows.")

print(f"The dataframe oneshot has {len(oneshot)} rows.")

print(f"The dataframe fewshot has {len(fewshot)} rows.")

print(f"The dataframe Nigel has {len(Nigel)} rows.")

Main.fillna(0, inplace=True)
Nigel.fillna(0, inplace=True)
Rebeca.fillna(0, inplace=True)
bad.fillna(0, inplace=True)
good.fillna(0, inplace=True)
cot.fillna(0, inplace=True)
#cove.fillna(0, inplace=True)
oneshot.fillna(0, inplace=True)
fewshot.fillna(0, inplace=True)

The dataframe Main has 400 rows.
The dataframe bad has 400 rows.
The dataframe good has 400 rows.
The dataframe cot has 400 rows.
The dataframe oneshot has 400 rows.
The dataframe fewshot has 400 rows.
The dataframe Nigel has 400 rows.


In [67]:
Main = Main.astype(int, errors='ignore')
Nigel = Nigel.astype(int, errors='ignore')
Rebeca = Rebeca.astype(int, errors='ignore')
bad = bad.astype(int, errors='ignore')
good = good.astype(int, errors='ignore')
cot = cot.astype(int, errors='ignore')
#cove = cove.astype(int, errors='ignore')
oneshot = oneshot.astype(int, errors='ignore')
fewshet = fewshot.astype(int, errors='ignore')

In [68]:
Main.head()

,Religious Community Involvement,Local Venue Visitation,Secular Volunteer Service,Online Social Networking,Workplace Socializing,Athletic Engagement,Cultural-Hobby Participation,Neighborhood Integration,Broader Event and Group Participation
0,0,0,1,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,1


In [69]:
association_measures = []
p_values = []
jaccard_similarities = []
cosine_similarities = []

for i in range(5):
    main_col = Main.columns[i]
    Rebeca_col = Rebeca.columns[i]

    # Create a contingency table
    contingency_table = pd.crosstab(Main[main_col], Rebeca[Rebeca_col])

    # Calculate the Chi-squared test
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    association_measures.append(chi2)  # chi2 as a measure of association
    p_values.append(p_value)

    # Calculate Jaccard Similarity
    intersection = sum((Main[main_col] == 1) & (Rebeca[Rebeca_col] == 1))
    union = sum((Main[main_col] == 1) | (Rebeca[Rebeca_col] == 1))
    jaccard_similarity = intersection / union if union != 0 else 0
    jaccard_similarities.append(jaccard_similarity)

Rebeca_df = pd.DataFrame({
    'Chi2_Main_Rebeca': association_measures,
    'P_value_Main_Rebeca': p_values,
    'Jaccard_Similarity': jaccard_similarities,
}, index=Main.columns[:5])

print(Rebeca_df)

                                 Chi2_Main_Rebeca  P_value_Main_Rebeca  \
Religious Community Involvement        349.100127         6.654271e-78   
Local Venue Visitation                 307.872519         6.348733e-69   
Secular Volunteer Service              350.438400         3.401512e-78   
Online Social Networking               343.179009         1.295831e-76   
Workplace Socializing                  345.060762         5.043710e-77   

                                 Jaccard_Similarity  
Religious Community Involvement            0.911765  
Local Venue Visitation                     0.820513  
Secular Volunteer Service                  0.923077  
Online Social Networking                   0.895833  
Workplace Socializing                      0.904762  


In [73]:
category_dfs = {
    'Nigel': Nigel,
    'Rebeca': Rebeca,
    'bad': bad,
    'good': good,
    'cot': cot,
    #'cove': cove,
    'oneshot': oneshot,
    'fewshot': fewshot
}
result_dfs = {}

for cat_name, cat_df in category_dfs.items():
    association_measures = []
    p_values = []
    jaccard_similarities = []

    for i in range(9):
        main_col = Main.columns[i]
        cat_col = cat_df.columns[i]

            # Create a contingency table
        contingency_table = pd.crosstab(Main[main_col], cat_df[cat_col])

            # Calculate the Chi-squared test
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        association_measures.append(chi2)  # chi2 as a measure of association
        p_values.append(p_value)

            # Calculate Jaccard Similarity
        intersection = sum((Main[main_col] == 1) & (cat_df[cat_col] == 1))
        union = sum((Main[main_col] == 1) | (cat_df[cat_col] == 1))
        jaccard_similarity = intersection / union if union != 0 else 0
        jaccard_similarities.append(jaccard_similarity)

            # Calculate Cosine Similarity
        dot_product = np.dot(Main[main_col], cat_df[cat_col])
        magnitude_main = np.sqrt(np.dot(Main[main_col], Main[main_col]))
        magnitude_cat = np.sqrt(np.dot(cat_df[cat_col], cat_df[cat_col]))

    # Create the DataFrame
    chi_squared_jaccard_cosine_df = pd.DataFrame({
        'Chi2_Main_Cat': association_measures,
        'P_value_Main_Cat': p_values,
        'Jaccard_Similarity': jaccard_similarities,
    }, index=Main.columns[:9])

    # Store the DataFrame in a dictionary with a unique key
    result_dfs[cat_name] = chi_squared_jaccard_cosine_df

In [74]:
result_dfs

{'Nigel':                                        Chi2_Main_Cat  P_value_Main_Cat  \
 Religious Community Involvement           246.172112      1.774104e-55   
 Local Venue Visitation                    211.747702      5.709184e-48   
 Secular Volunteer Service                 189.629630      3.829793e-43   
 Online Social Networking                  282.641653      1.995003e-63   
 Workplace Socializing                     203.783029      3.121246e-46   
 Athletic Engagement                       273.532341      1.927832e-61   
 Cultural-Hobby Participation               84.656763      3.549279e-20   
 Neighborhood Integration                  171.897872      2.848822e-39   
 Broader Event and Group Participation      28.746792      8.248646e-08   
 
                                        Jaccard_Similarity  
 Religious Community Involvement                  0.690476  
 Local Venue Visitation                           0.615385  
 Secular Volunteer Service                        0.5476

In [75]:
# Initialize an empty dictionary to hold the average Jaccard Similarity for each category
average_jaccard_similarities = {}

# Iterate over each item in result_dfs to calculate the average Jaccard Similarity
for cat_name, df in result_dfs.items():
    average_jaccard_similarities[cat_name] = df['Jaccard_Similarity'].mean()

# Convert the dictionary of averages into a DataFrame
# The keys become the column names, and the values become the row values
average_jaccard_df = pd.DataFrame(average_jaccard_similarities, index=["Average_Jaccard_Similarity"])

print(average_jaccard_df)

                               Nigel    Rebeca       bad      good       cot  \
Average_Jaccard_Similarity  0.560135  0.808843  0.735932  0.739812  0.753205   

                             oneshot   fewshot  
Average_Jaccard_Similarity  0.730152  0.730687  


In [76]:
average_jaccard_df_transposed = average_jaccard_df.T.rename(columns={"Average_Jaccard_Similarity": "Average_Jaccard"})

In [77]:
average_jaccard_df_transposed.to_csv('data/a19f/average_jaccard.csv')

In [78]:
# Initialize an empty DataFrame to hold the Jaccard Similarity columns
jaccard_similarity_df = pd.DataFrame()

# Iterate over the result_dfs dictionary to extract the Jaccard Similarity column
for cat_name, df in result_dfs.items():
    # Add the extracted column to the new DataFrame with a new column name
    jaccard_similarity_df[cat_name] = df['Jaccard_Similarity']

print(jaccard_similarity_df)

                                          Nigel    Rebeca       bad      good  \
Religious Community Involvement        0.690476  0.911765  1.000000  0.942857   
Local Venue Visitation                 0.615385  0.820513  0.702128  0.750000   
Secular Volunteer Service              0.547619  0.923077  0.857143  0.961538   
Online Social Networking               0.769231  0.895833  0.875000  0.833333   
Workplace Socializing                  0.653595  0.904762  0.746154  0.783333   
Athletic Engagement                    0.731707  0.761905  0.911111  0.869565   
Cultural-Hobby Participation           0.353846  0.533333  0.400000  0.380000   
Neighborhood Integration               0.542373  0.800000  0.734694  0.739130   
Broader Event and Group Participation  0.136986  0.728395  0.397163  0.398551   

                                            cot   oneshot   fewshot  
Religious Community Involvement        0.942857  0.971429  0.942857  
Local Venue Visitation                 0.809524  

In [79]:
jaccard_similarity_df.to_csv('data/a19f/similiarity_jaccard.csv')